In [1]:
from hydra import compose, initialize
import hydra, os
from glob import glob
from omegaconf import DictConfig, OmegaConf

In [2]:
config_path = 'config'
with initialize(config_path=config_path,version_base=None):
    cfg = compose(config_name="baseline.yaml")

In [40]:
config_path = 'config'
with initialize(config_path=config_path,version_base=None):
    cfg = compose(config_name="baseline.yaml")
    signals = glob(os.path.join(cfg.data.signals_path, '*.yaml'))
    signals = {os.path.basename(conf).split('.')[0]: "/".join(conf.split('\\')[-4:]) for conf in signals}
    signal = compose(config_name=signals['square'])
    cfg.data.exo_input.signals.state_0 = signal
    print(OmegaConf.to_yaml(cfg,resolve=True))

seed: 0
data:
  data_dir: data
  dataset_name: duffing
  signals_path: config\data\exo_input\signals
  ds_path: ''
  pinn_sampling: split_traj
  gen_mode: backward
  validation:
    method: horizon
    time: 10
  sim_time:
    _target_: src.simulators.types.SimTime
    t0: 0.0
    tn: 50.0
    eps: 0.05
  solver:
    _target_: src.simulators.solvers.get_solver
    name: rk4
  ph_sampler:
    _target_: src.simulators.sampler.get_sampler
    sampler_type: lhs
    seed: 8888
    sample_space:
    - - -1
      - 1
    - - -1
      - 1
  dataloader:
    seed: 0
    batch_size: 32
    shuffle: true
  exo_input:
    _target_: src.simulators.exogenous_signals.SignalGenerator
    num_samples: 5
    seed: 0
    sys_states: 2
    signals:
      state_0:
        data:
          exo_input:
            signals:
              _target_: src.simulators.exogenous_signals.AlternatingSquareSignal
              seed: 0
              num_samples: 5
              params:
                _target_: src.simulat

True